In [ ]:
!pip3 uninstall keras-nightly
!pip3 install h5py==2.10.0
# instalacja poprzednich wersji tensorflow i keras
%tensorflow_version 1.x
import tensorflow
!pip3 uninstall keras
!pip install q keras==2.2.4
import keras
import glob
import cv2 as cv

TensorFlow 1.x selected.
Found existing installation: Keras 2.2.4
Uninstalling Keras-2.2.4:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/Keras-2.2.4.dist-info/*
    /usr/local/lib/python3.7/dist-packages/docs/*
    /usr/local/lib/python3.7/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.7/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.2.4
  Using cached Keras-2.2.4-py2.py3-none-any.whl (312 kB)


Using TensorFlow backend.


In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git
%cd Mask_RCNN
!python setup.py install
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn import visualize
import os
import skimage

from google.colab import drive
drive._mount('/content/drive')

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 125.23 MiB | 29.81 MiB/s, done.
Resolving deltas: 100% (565/565), done.
/content/Mask_RCNN
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running i

In [ ]:
class_id_to_name_mapping = {
    0: 'BG',
    1: 'plane',
    2: 'ship',
    3: 'storage-tank',
    4: 'baseball-diamond',
    5: 'tennis-court',
    6: 'basketball-court',
    7: 'ground-track-field',
    8: 'harbor',
    9: 'bridge',
    10: 'large-vehicle',
    11: 'small-vehicle',
    12: 'helicopter',
    13: 'roundabout',
    14: 'soccer-ball-field',
    15: 'swimming-pool',
    16: 'container-crane'
}

In [ ]:
class Tests:
    def __init__(self, model, img_format, folder_num, display=False):
        # zmienne
        self.model = model
        self.img_format = img_format
        self.folder_num = folder_num
        self.display = display
        # zdefiniowanie modelu
        self.rcnn = MaskRCNN(mode='inference', model_dir='./', config=self.SA_Config())
        # zaladowanie wag modelu
        self.rcnn.load_weights(self.model, by_name=True)

    # konfiguracja do detekcji
    class SA_Config(Config):
        NAME = "sat_aerial_object_detection"
        BACKBONE = 'resnet101'
        GPU_COUNT = 1
        NUM_CLASSES = 1 + 16
        DETECTION_MIN_CONFIDENCE = 0.68
        STEPS_PER_EPOCH = 1000
        BATCH_SIZE = 1
        IMAGES_PER_GPU = 1
        VALIDATION_STEPS = 80
        RPN_ANCHOR_SCALES = (10, 20, 40, 80, 160)
        TRAIN_ROIS_PER_IMAGE = 700
        DETECTION_MAX_INSTANCES = 1600

    # glowna petla programu
    def ocr(self):
        # lista obrazow
        list_of_imgs = glob.glob(f'/content/drive/MyDrive/DOTA/Dataset/Val/mod/images{self.folder_num}' + f'/*.{self.img_format}')
        print(len(list_of_imgs))
        for index, img in enumerate(list_of_imgs):
            # zaladowanie kolejnego obrazu z folderu
            image = skimage.io.imread(img)
            # predykcja
            results = self.rcnn.detect([image], verbose=0)
            print(round((index/len(list_of_imgs) * 100), 2), '%')
            # uzyskane wyniki -> wspolrzedne krancow tablicy
            # zapis uzyskanych wynikow -> lista 4-elementowych wektorow
            r = results[0]
            self.save_results(r['rois'], r['scores'], r['class_ids'], self.folder_num, img)
            if self.display:
                visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], list(class_id_to_name_mapping.values()), r['scores'])
        
    def save_results(self, bounding_boxes, scores, classes, folder_num, img_name):
        assert len(bounding_boxes) == len(scores) == len(classes)
        bb_file_name = os.path.basename(img_name).replace(f'.{self.img_format}', '.txt')
        with open(f'/content/drive/MyDrive/DOTA/Dataset/results_mask_rcnn/{folder_num}/{bb_file_name}', 'w') as file:
            for bounding_box, score, class_ in zip(bounding_boxes, scores, classes):
                xs = [bounding_box[1], bounding_box[3]]
                ys = [bounding_box[0], bounding_box[2]]
                file.write(f'{class_id_to_name_mapping[class_]} {score} {int(min(xs))} {int(min(ys))} {int(max(xs))} {int(max(ys))}\n')

In [ ]:
fn = 5
testy = Tests(model='/content/drive/MyDrive/DOTA/Out_model/sat_aerial_object_detection20211001T1246/mask_rcnn_sat_aerial_object_detection_0180.h5', 
              img_format='png',
              folder_num=fn,
              display=False
              )
testy.ocr()
print(len(glob.glob(f'/content/drive/MyDrive/DOTA/Dataset/results_mask_rcnn/{fn}/*.txt')))








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.






Re-starting from epoch 180
1797
0.0 %
0.06 %
0.11 %
0.17 %
0.22 %
0.28 %
0.33 %
0.39 %
0.45 %
0.5 %
0.56 %
0.61 %
0.67 %
0.72 %
0.78 %
0.83 %
0.89 %
0.95 %
1.0 %
1.06 %
1.11 %
1.17 %
1.22 %
1.28 %
1.34 %
1.39 %
1.45 %
1.5 %
1.56 %
1.61 %
1.67 %
1.73 %
1.78 %
1.84 %
1.89 %
1.95 %
2.0 %
2.06 %
2.11 %
2.17 %
2.23 %
2.28 %
2.34 %
2.39 %
2.45 %
2.5 %
2.56 %
2.62 %
2.67 %
2.73 %
2.78 %
2.84 %
2.89 %
2.95 %
3.01 %
3.06 %
3.12 %
3.17 %
3.23 %
3.28 %
3.34 %
3.39 %
3.45 %
3.51 %
3.56 %
3.62 %
3.67 %
3.73 %
3.78 %
3.84 %
3.9 %
3.95 %
4.01 %
4.06 %
4.12 %
4.17 %
4.23 %
4.28 %
4.34 %
4.4 %
4.45 %
4.51 %
4.56 %
4.62 %
4.67 %
4.73 %
4.79 %
4.84 %
4.9 %
4.95 %
5.01 %
5.06 %
5.12 %
5.18 %
5.23 %
5.29 %
5.34 %
5.4 %
5.45 %
5.51 %
5.56 %
5.62 %
5.68 %
5.73 %
5.79 %
5.84 %
5.9